# Demonstration of brute force optimisation by looping

As well as using the genetic algorithm optimiser (see the [Optimisation_demo notebook](Optimisation_demo.ipynb)) it is possible to do brute force optimisation by explicitly looping through a range of parameter values, running the facility for each, and seeing which gives the best outcome. This notebook contains an example of how to do so.

As is common by now, we start by loading the appropriate parts of the PyBioPharma framework, and set up the facility we want to optimise. See the [introductory demo notebook](User_demo.ipynb) for more explanation about this section. As in that example, we load default parameters from the [data](./data) folder by specifying it as the `data_path` argument to `Facility`.

In [ ]:
import biopharma as bp

facility = bp.Facility(data_path='data')

# Define the steps needed to create our single product
from biopharma.process_steps import (
    
)
steps = [
    
]
product = bp.Product(facility, steps)

We then select which step we are going to optimise. We could do this by `step = steps[4]` but selecting by name is more robust.

In [ ]:
step = product.sequence.findStep('test step')

Now we loop over parameter values. There is quite a lot going on in this cell, so this explanation is line-by-line.
1. The `results` list is going to keep track of both the varying parameter values and resulting outputs of interest.
2. The first line defines what values to loop over. The `range` function defines an initial value, an end value, and an increment. Note that this defines a half open interval: the initial value is included in the list of values, but the end value is not. So this will make the `magnitude` variable take each of the values 170, 180, 190, 200, 210, 220, 230, 240, 250, 260, 270 in turn.
3. The `range` function can only iterate over numbers (integers or floating point) but our parameters are quantities with units. We therefore define `param` with the correct units.
4. We reset all facility parameters by loading from file, just in case a previous run changed any of them.
5. We set the parameter that we are varying.
6. Running the model is done within a `try-except` block, which will catch any errors thrown by running the model. This is important because some parameter values may result in the facility not being able to produce any product, which will cause an error. When this happens, the last line will print a message, but the loop will keep running.
7. The facility model is run.
8. Assuming there were no errors, the results of this run are appended to the `results` list as a dictionary.

In [ ]:
results = []
for magnitude in range(170, 280, 10):
    param = magnitude * bp.units('cm/hour')
    facility.load_parameters()
    step.parameters['param'] = param
    try:
        facility.run()
        results.append({'Parameter': param,
                        'Output': product.outputs['output']})
    except Exception as e:
        print('Failed to run with parameter {}'.format(param))

The final step is to report on the results. To make sorting and display easy we convert the `results` list into a [pandas](http://pandas.pydata.org/) DataFrame. The final line then sorts the table by the column labelled 'Output' and displays it.

In [ ]:
import pandas as pd
results = pd.DataFrame(results)
print('Results in increasing order:')
results.sort_values(by='Output')